# 豆瓣TOP250短评数据建立情感分析器

In [1]:
import re
import pandas as pd
import jieba

* 读取文件中的评论和相应的用户评分
* 将用户评分转换为数值型
* 取出评分不等于3的评论和评分
* 将低于3分的标记为0(不喜欢)，高于3分的标记为1(喜欢)

In [2]:
# 读取文件短评内容
with open('clean_data.csv', 'r') as f:
    comment_data = f.read()
comment_data = comment_data.split('\n') # 以换行符分割短评

# 用短评和用户评分创建DataFrame
comment = []
user_score = []
for item in comment_data:
    s = re.search(r'(\w+,)(https://www.douban.com/people/.+/,)(.*,)(\d,)(\d{4}-\d{2}-\d{2} \d{2}:\d{2}:\d{2},)(.+)(,\d+)', item)
    if s:
        comment.append(s.group(6)) # 取出短评信息中的评论
        user_score.append(s.group(4)[:-1]) # 取出短评信息中的用户评分
comments = pd.DataFrame({'comment':comment,
                        'user_score':user_score})

# 将低于3分的标记为0(不喜欢)，高于3分的标记为1(喜欢)
def comment_type(score):
    if score > 3:
        return 1
    else:
        return 0
    
comments['user_score'] = pd.to_numeric(comments['user_score']) # 转换成数值型
# 取出用户评分不为3的数据
comments_for_model = comments.loc[comments['user_score'] != 3].copy()
comments_for_model['user_score'] = comments_for_model['user_score'].apply(comment_type)
# 查看数据前5行
comments_for_model.head()

,comment,user_score
1,我们大天朝每天都在发生这样的事儿，类似的纪录片可以看到很多，只是没进入大屏幕而已啦。说实话，...,0
2,无恶意…只是本人对记录片真心无爱…看了75分钟算是我看的时间最久的纪录片了！,0
4,其实我觉得这部拍得不错了。,1
5,时间走过70年的2013年元旦， 我第一次看了这部如雷贯耳的经典老片。 作为经典， 它当之无...,1
6,更爱库布里克袅。难以形容的愉悦与快感，一种脑壳难受三观颠倒的快感，一种投身邪恶告别俗人的愉悦...,1


去除评论中的数字和字母，并去掉全为英文的评论，只保留中文的评论

In [3]:
clean_comment = comments_for_model['comment'].str.replace(r'[a-zA-Z0-9]', '') # 去掉评论中的字母和数字
cn_comments = clean_comment.str.contains(r'\w') # 中文评论的索引
model_data = comments_for_model[cn_comments].copy() # 取出中文评论
model_data['comment'] = model_data['comment'].str.replace(r'[a-zA-Z0-9]', '') # 去掉评论中的字母和数字
model_data.head()

,comment,user_score
1,我们大天朝每天都在发生这样的事儿，类似的纪录片可以看到很多，只是没进入大屏幕而已啦。说实话，...,0
2,无恶意…只是本人对记录片真心无爱…看了分钟算是我看的时间最久的纪录片了！,0
4,其实我觉得这部拍得不错了。,1
5,时间走过年的年元旦， 我第一次看了这部如雷贯耳的经典老片。 作为经典， 它当之无愧， 算作新...,1
6,更爱库布里克袅。难以形容的愉悦与快感，一种脑壳难受三观颠倒的快感，一种投身邪恶告别俗人的愉悦...,1


In [4]:
# 对评论进行分词
model_data['comment'] = model_data['comment'].apply(lambda x: ' '.join(jieba.cut(x)))

X = model_data['comment']
y = model_data['user_score']
y.value_counts()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 1.331 seconds.
Prefix dict has been built succesfully.


1    135104
0     88277
Name: user_score, dtype: int64

可以看到表示喜欢的评论多于表示不喜欢的评论

In [5]:
# 读取停用词表
stopwords = []
with open('stopwords.txt', 'r', encoding='utf-8') as f:
    for stopword in f.readlines():
        stopwords.append(stopword[:-1]) # 去掉换行符
stopwords[0] = stopwords[0][-1] # 去除第一个字符中的无用字符
stopwords = stopwords + ['一个', '这部', '一部', '片子', '这是', '还有', '应该']

In [6]:
from sklearn.model_selection import train_test_split
# 将数据集分割为训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

### 朴素贝叶斯

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# 抽取1-gram和2-gram的统计特征
vec = CountVectorizer(analyzer='word', ngram_range=(1, 2), max_features=8000, stop_words=stopwords)
vec.fit(X_train)
classifier = MultinomialNB() # 创建实例
classifier.fit(vec.transform(X_train), y_train) # 拟合数据
classifier.score(vec.transform(X_test), y_test) # 模型准确率

0.79164129928732585

In [9]:
from sklearn.metrics import confusion_matrix, classification_report
# 预测测试集
y_predict = classifier.predict(vec.transform(X_test))
print (confusion_matrix(y_predict, y_test)) # 输出混淆矩阵
print (classification_report(y_predict, y_test)) # 输出其它评价指标

[[15188  4725]
 [ 6911 29022]]
             precision    recall  f1-score   support

          0       0.69      0.76      0.72     19913
          1       0.86      0.81      0.83     35933

avg / total       0.80      0.79      0.79     55846



### 逻辑回归

In [16]:
from sklearn.linear_model import LogisticRegression

# 抽取1-gram和2-gram的统计特征
vec = CountVectorizer(analyzer='word', ngram_range=(1, 2), max_features=1000, stop_words=stopwords)
vec.fit(X_train)

lr_clf = LogisticRegression() # 创建实例
lr_clf.fit(vec.transform(X_train), y_train) # 拟合数据
lr_clf.score(vec.transform(X_test), y_test) # 模型准确率

0.76044837589084269

对模型进行网格搜索

In [14]:
from sklearn.model_selection import GridSearchCV

# 网格搜索的参数
param_grid = {'penalty':['l1', 'l2'],
             'C':[0.001, 0.01, 0.1, 1, 10, 100]}
best_score = 0
# 用不同特征数进行网格搜索
for features in range(1000, 5500, 500):
    # 抽取1-gram和2-gram的统计特征
    vec = CountVectorizer(analyzer='word', ngram_range=(1, 2), max_features=features, stop_words=stopwords)
    vec.fit(X_train)
    # 进行网格搜索
    grid_search = GridSearchCV(LogisticRegression(), param_grid=param_grid, cv=5).fit(vec.transform(X_train), y_train)
    estimator = grid_search.best_estimator_
    score = grid_search.best_score_
    params = grid_search.best_params_
    # 取出准确路最佳的模型
    if score > best_score:
        best_estimator = estimator
        best_score = score
        best_params = params

In [23]:
print (best_estimator) # 输出最佳模型
print (best_score) # 输出最佳准确率
print (best_params) # 输出最佳模型的参数
# 预测测试集
y_predict = best_estimator.predict(vec.transform(X_test))
print (confusion_matrix(y_predict, y_test)) # 输出混淆矩阵
print (classification_report(y_predict, y_test)) # 输出其它评价指标

LogisticRegression(C=0.1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
0.799164353717
{'penalty': 'l2', 'C': 0.1}
[[14373  3296]
 [ 7726 30451]]
             precision    recall  f1-score   support

          0       0.65      0.81      0.72     17669
          1       0.90      0.80      0.85     38177

avg / total       0.82      0.80      0.81     55846



用网格搜索得到的最佳模型的准确率略高于朴素贝叶斯，但是使用特征数少于朴素贝叶斯所使用的特征数

### 用TfidfVectorizer提取文本特征并用朴素贝叶斯建模

In [8]:
from sklearn.feature_extraction.text import  TfidfVectorizer
# 抽取1-gram和2-gram的统计特征
tf = TfidfVectorizer(analyzer='word',
                    ngram_range=(1, 2),
                     max_features=8000,
                    stop_words=stopwords)
tf.fit(X_train)
# 建模
tf_clf = MultinomialNB()
tf_clf.fit(tf.transform(X_train), y_train)
tf_clf.score(tf.transform(X_test), y_test)

0.79916198116248249

In [11]:
y_predict = tf_clf.predict(tf.transform(X_test))
print (confusion_matrix(y_predict, y_test))
print (classification_report(y_predict, y_test))

[[13768  2885]
 [ 8331 30862]]
             precision    recall  f1-score   support

          0       0.62      0.83      0.71     16653
          1       0.91      0.79      0.85     39193

avg / total       0.83      0.80      0.81     55846

